In [102]:
import pandas as pd

In [103]:
movies=pd.read_csv("tmdb_5000_movies.csv")

In [104]:
credits=pd.read_csv("tmdb_5000_credits.csv")

In [105]:
new_movies=movies.merge(credits,on="title")

In [106]:
new_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [107]:
#genre
#id
#keyword
#overview
#title
#cast
#crew these are the only columns we need

In [108]:
new_movies=new_movies[['id','title','genres','keywords','overview','cast','crew']]
new_movies.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [109]:
new_movies.dropna(inplace=True)

In [110]:
import json

In [111]:
def formatgenre(text):
    genres=[]
    for i in json.loads(text):
        genres.append(i["name"])
    return genres

In [112]:
new_movies['genres']=new_movies['genres'].apply(formatgenre)

In [113]:
def formatkeywords(text):
    keywords=[]
    for i in json.loads(text):
        keywords.append(i["name"])
    return keywords

In [114]:
new_movies['keywords']=new_movies['keywords'].apply(formatkeywords)

In [115]:
def castformatter(text):
    cast=[]
    z=0
    for i in json.loads(text):
        if z==3:
            break
        z=z+1
        cast.append(i["name"])
    return cast

In [116]:
new_movies['cast']=new_movies['cast'].apply(castformatter)

In [117]:
def crewformatter(text):
    director=[]
    for i in json.loads(text):
        if i["job"]=="Director":
            director.append(i["name"])
            break
    return director

In [118]:
new_movies['crew']=new_movies['crew'].apply(crewformatter)

In [119]:
new_movies["keywords"][0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [120]:
new_movies["overview"]=new_movies["overview"].apply(lambda x: x.split())
new_movies['crew']=new_movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies['genres']=new_movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies['cast']=new_movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies['keywords']=new_movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
new_movies["allkeywords"]=new_movies["overview"]+new_movies["keywords"]+new_movies["genres"]+new_movies["cast"]+new_movies["crew"]

In [121]:
movies=new_movies[["id","title","allkeywords"]]

In [122]:
movies["allkeywords"]=movies["allkeywords"].apply(lambda x:" ".join(x))

C:\Users\drenergydrink\AppData\Local\Temp\ipykernel_22004\2329735072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["allkeywords"]=movies["allkeywords"].apply(lambda x:" ".join(x))


In [123]:
movies.head()

,id,title,allkeywords
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [124]:
movies["allkeywords"]=movies["allkeywords"].apply(lambda x:x.lower())

C:\Users\drenergydrink\AppData\Local\Temp\ipykernel_22004\3099174844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["allkeywords"]=movies["allkeywords"].apply(lambda x:x.lower())


In [125]:
movies["allkeywords"][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver jamescameron'

In [128]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")

In [129]:
vectors=cv.fit_transform(movies["allkeywords"]).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)